# AlphaPept Timing Check

This notebook will run several testfiles and test the timing.

In [ ]:
import os
import alphapept.settings
import sys
sys.path.append(os.path.join(".."))
import test_ci

try:
    import wget
except ImportError:
    !pip install wget
    import wget
    
tmp_folder = os.path.join(os.getcwd(),'temp/')


BASE_DIR = os.path.join(tmp_folder , 'test_files/') # Storarge location for test files
TEST_DIR = os.path.join(tmp_folder, 'test_temp/')
ARCHIVE_DIR = os.path.join(tmp_folder, ' test_archive/')

MONGODB_USER = ''
MONGODB_URL = ''

if not os.path.isdir(tmp_folder):
    os.mkdir(tmp_folder)

test_ci.config_test_paths(BASE_DIR, TEST_DIR, ARCHIVE_DIR, MONGODB_USER, MONGODB_URL)

timing_check = {}

def get_timings():
    settings = alphapept.settings.load_settings(os.getcwd() + '/temp/test_temp/results.yaml')

    total = settings['summary']['timing']['total (min)']
    total_no_db = total - settings['summary']['timing']['create_database (min)']
    total_no_db_ff_raw = total_no_db - settings['summary']['timing']['feature_finding (min)'] - settings['summary']['timing']['import_raw_data (min)'] 
    
    return total, total_no_db, total_no_db_ff_raw

## Single Files

Compare to the single HeLa test files

### Thermo IRT

In [ ]:
%run -i ../test_ci.py $tmp_folder thermo_irt

try:
    timing_check['thermo_irt'] = get_timings()
except Exception as e:
    pass

### Thermo HeLa

In [ ]:
%run -i ../test_ci.py $tmp_folder thermo_hela


try:
    timing_check['thermo_hela'] = get_timings()
except Exception as e:
    pass

## PXD006109

In [ ]:
%run -i ../test_ci.py $tmp_folder PXD006109 

try:
    timing_check['PXD006109'] = get_timings()
except Exception as e:
    pass

## Bruker HeLa

In [ ]:
%run -i ../test_ci.py $tmp_folder bruker_hela

try:
    timing_check['bruker_hela'] = get_timings()
except Exception as e:
    pass

## PXD010012

In [ ]:
%run -i ../test_ci.py $tmp_folder PXD010012

try:
    timing_check['PXD010012'] = get_timings()
except Exception as e:
    pass

In [ ]:
timing_check

In [ ]:
df = pd.DataFrame(timing_check).T

df.columns = ['Full', 'Precomputed database', 'Precomputed database, feature finding and raw import']
display(df)